[View in Colaboratory](https://colab.research.google.com/github/manan-arya/Analytics_Vidhya_Competition/blob/master/XGBoost_CV.ipynb)

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [2]:
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
!wget https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv

--2018-09-02 06:11:13--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/train_NIR5Yl1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12564851 (12M) [text/plain]
Saving to: ‘train_NIR5Yl1.csv’

train_NIR5Yl1.csv   100%[===================>]  11.98M  21.7MB/s    in 0.6s    

2018-09-02 06:11:14 (21.7 MB/s) - ‘train_NIR5Yl1.csv’ saved [12564851/12564851]

--2018-09-02 06:11:15--  https://raw.githubusercontent.com/manan-arya/Analytics_Vidhya_Competition/master/test_8i3B3FC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response

In [0]:
train=pd.read_csv("train_NIR5Yl1.csv")
test=pd.read_csv("test_8i3B3FC.csv")

In [4]:
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0


In [5]:
test.head()

,ID,Tag,Reputation,Answers,Username,Views
0,366953,a,5645.0,3.0,50652,33200.0
1,71864,c,24511.0,6.0,37685,2730.0
2,141692,i,927.0,1.0,135293,21167.0
3,316833,i,21.0,6.0,166998,18528.0
4,440445,i,4475.0,10.0,53504,57240.0


In [6]:
train = pd.concat([train,pd.get_dummies(train['Tag'], drop_first=True)], axis=1)
train.head()

,ID,Tag,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,52664,a,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,468453,c,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,96996,a,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,131465,c,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [7]:
test = pd.concat([test,pd.get_dummies(test['Tag'], drop_first=True)], axis=1)
test.head()

,ID,Tag,Reputation,Answers,Username,Views,c,h,i,j,o,p,r,s,x
0,366953,a,5645.0,3.0,50652,33200.0,0,0,0,0,0,0,0,0,0
1,71864,c,24511.0,6.0,37685,2730.0,1,0,0,0,0,0,0,0,0
2,141692,i,927.0,1.0,135293,21167.0,0,0,1,0,0,0,0,0,0
3,316833,i,21.0,6.0,166998,18528.0,0,0,1,0,0,0,0,0,0
4,440445,i,4475.0,10.0,53504,57240.0,0,0,1,0,0,0,0,0,0


In [0]:
train.drop("Tag",axis=1,inplace=True)
test.drop("Tag",axis=1,inplace=True)


In [9]:
train.head()

,ID,Reputation,Answers,Username,Views,Upvotes,c,h,i,j,o,p,r,s,x
0,52664,3942.0,2.0,155623,7855.0,42.0,0,0,0,0,0,0,0,0,0
1,327662,26046.0,12.0,21781,55801.0,1175.0,0,0,0,0,0,0,0,0,0
2,468453,1358.0,4.0,56177,8067.0,60.0,1,0,0,0,0,0,0,0,0
3,96996,264.0,3.0,168793,27064.0,9.0,0,0,0,0,0,0,0,0,0
4,131465,4271.0,4.0,112223,13986.0,83.0,1,0,0,0,0,0,0,0,0


In [0]:
X=train.drop("Upvotes",axis=1)
y=train["Upvotes"]


In [0]:
X_train, X_test, y_train, y_test= train_test_split(X, y,test_size=0.3, random_state=123)

In [19]:
DM_train = xgb.DMatrix(data=X_train,label=y_train)
DM_test = xgb.DMatrix(data=X_test,label=y_test)
params = {"booster":"gblinear","objective":"reg:linear"}
xg_reg = xgb.train(params = params, dtrain=DM_train,num_boost_round=200)

preds = xg_reg.predict(DM_test)
rmse = np.sqrt(mean_squared_error(y_test,preds))
print("RMSE: %f" % (rmse))



RMSE: 2742.777604


In [20]:
review_data_matrix=xgb.DMatrix(data=X,label=y)

untuned_params = {"objective":"reg:linear"}

tuned_cv_rmse = xgb.cv(dtrain = review_data_matrix,params=untuned_params, nfold=7, num_boost_round=200, metrics="rmse",as_pandas=True, seed=123)

print(type(tuned_cv_rmse))

print("Tuned rmse: %f" %((tuned_cv_rmse["test-rmse-mean"]).tail(1)))


<class 'pandas.core.frame.DataFrame'>
Tuned rmse: 1356.342329


In [21]:
new_preds=tuned_cv_rmse.predict(X_test)
new_rmse = np.sqrt(mean_squared_error(y_test,new_preds))
print("RMSE: %f" % (new_rmse))


AttributeError: ignored

In [26]:
print("\nSetting up data for XGBoost ...")
# xgboost params
xgb_params = {
    #'eta': 0.037,
    'max_depth': 5,
    'subsample': 0.80,
    'objective': 'reg:linear',
    #'eval_metric': 'mae',
    #'lambda': 0.8,   
    #'alpha': 0.4, 
    #'silent': 1
}

# AttributeError: module 'xgboost' has no attribute 'DMatrix'
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

num_boost_rounds = 200
print("num_boost_rounds="+str(num_boost_rounds))

# train model
print( "\nTraining XGBoost ...")
model = xgb.train(dict(xgb_params, silent=1), dtrain, 
num_boost_round=num_boost_rounds)

print( "\nPredicting with XGBoost ...")
xg_preds = model.predict(dtest)

rmse = np.sqrt(mean_squared_error(y_test,xg_preds))
print("RMSE: %f" % (rmse))


Setting up data for XGBoost ...
num_boost_rounds=200

Training XGBoost ...

Predicting with XGBoost ...
RMSE: 1364.225568
